In [13]:
import numpy as np
import os
import shutil

In [14]:
#Set input, output and quadrature mode

#COSY MC 
#magritek_data_path='example/COSY/'
#file_path_fid = magritek_data_path+'cosyFID.pt2'
#destination_folder = 'example/1001'
#quad=1 #1 for MC, 2 for States, 3 for Echo-Antiecho

#HSQC MC 
#magritek_data_path='example/HSQC_MC/'
#file_path_fid = magritek_data_path+'HSQC_FID.pt2'
#destination_folder = 'example/1002'
#quad=1 #1 for MC, 2 for States, 3 for Echo-Antiecho

#HSQC States
magritek_data_path='example/HSQCtgME/'
file_path_fid = magritek_data_path+'HSQCtgME_FID.pt2'
destination_folder = 'example/1003'
quad=2 #1 for MC, 2 for States, 3 for Echo-Antiecho

output_binary_file = destination_folder+'/ser'
source_template_folder = 'template'
header_size_fid = 3386


with open(file_path_fid, 'rb') as f:
    binary_data_fid = f.read()
binary_data_truncated_fid = binary_data_fid[header_size_fid:]


data_array_fid = np.frombuffer(binary_data_truncated_fid, dtype=np.complex64)


In [15]:
def convert_numeric_strings(data_dict):
    """
    Converts string values in a dictionary to integers or floats if they represent numbers.

    Args:
        data_dict: The dictionary to process.

    Returns:
        A new dictionary with numeric strings converted to appropriate types.
    """
    converted_dict = {}
    for key, value in data_dict.items():
        if isinstance(value, str):
            try:
                if '.' in value:
                    converted_dict[key] = float(value)
                else:
                    converted_dict[key] = int(value)
            except ValueError:
                # If conversion fails, keep the original string value
                converted_dict[key] = value
        else:
            # Keep non-string values as they are
            converted_dict[key] = value
    return converted_dict

In [16]:
magritek_params = dict()
magritek_proc_params = dict()
with open(magritek_data_path +  'acqu.par') as f:
        for line in f:
            eq_index = line.find('=')
            var_name = line[:eq_index].strip()
            number = (line[eq_index + 1:].strip())
            if line.startswith('nucleus'):
                nuclei = line.split('=')[1].strip().replace('"', '').split('-')
                magritek_params['nucleus1'] = nuclei[0] if len(nuclei) > 0 else ''
                magritek_params['nucleus2'] = nuclei[1] if len(nuclei) > 1 else ''
            else:
                magritek_params[var_name] = number
if os.path.isfile(magritek_data_path +'proc.par'):
        with open(magritek_data_path + 'proc.par') as f:
            for line in f:
                eq_index = line.find('=')
                var_name = line[:eq_index].strip()
                number = (line[eq_index + 1:].strip())
                magritek_proc_params[var_name] = number

magritek_params=convert_numeric_strings(magritek_params)
magritek_proc_params=convert_numeric_strings(magritek_proc_params)

In [17]:
dim2_size = magritek_params['nrPnts']
dim1_size = magritek_params['nrSteps']*quad
data_2d_fid = data_array_fid.reshape((dim1_size, dim2_size))
data_to_save_complex = data_2d_fid

interleaved_data = np.empty((data_to_save_complex.shape[0], data_to_save_complex.shape[1], 2), dtype=np.float64)
interleaved_data[:, :, 0] = np.real(data_to_save_complex)
interleaved_data[:, :, 1] = np.imag(data_to_save_complex)


data_to_save_binary = interleaved_data.flatten()

data_to_save_binary = data_to_save_binary.astype(np.float64)

In [18]:
if os.path.exists(destination_folder):
    shutil.rmtree(destination_folder)
shutil.copytree(source_template_folder, destination_folder)

print(f"Folder '{source_template_folder}' copied to '{destination_folder}'")

data_to_save_binary.tofile(output_binary_file)

print(f"Time-domain data (FID) saved to {output_binary_file}.")

acqu_file_path =  destination_folder+'/acqu'
acqus_file_path = destination_folder+'/acqus'

with open(acqu_file_path, 'r') as f:
    acqu_content = f.readlines()

# Modify the content of the 'acqu' file
modified_acqu_content = []
for line in acqu_content:
    if line.startswith('##$TD='):
        modified_acqu_content.append(f'##$TD= {data_to_save_complex.shape[1]*2}\n')
    elif line.startswith('##$DIGTYP='):
        modified_acqu_content.append('##$DIGTYP= 17\n')
    elif line.startswith('##$DTYPA='):
        modified_acqu_content.append('##$DTYPA= 2\n')
    elif line.startswith('##$NUC1=') and 'nucleus1' in magritek_params:
        modified_acqu_content.append(f'##$NUC1= <{magritek_params["nucleus1"]}>\n')
    elif line.startswith('##$NUC2=') and 'nucleus2' in magritek_params:
        modified_acqu_content.append(f'##$NUC2= <{magritek_params["nucleus2"]}>\n')
    elif line.startswith('##$SFO1=') and 'b1Freq' in magritek_params:
        modified_acqu_content.append(f'##$SFO1= {magritek_params["b1Freq"]}\n')
    elif line.startswith('##$SFO2=') and 'b1Freq2' in magritek_params:
        modified_acqu_content.append(f'##$SFO2= {magritek_params["b1Freq2"]}\n')
    elif line.startswith('##$BF1=') and 'b1Freq' and 'centerFreqPPM'  in magritek_params and magritek_params["nucleus1"]==magritek_params["nucleus2"]:
        modified_acqu_content.append(f'##$BF1= {magritek_params["b1Freq"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$BF2=') and 'b1Freq2'  and 'centerFreqPPM'  in magritek_params and magritek_params["nucleus1"]==magritek_params["nucleus2"]:
        modified_acqu_content.append(f'##$BF2= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$BF1=') and 'b1Freq' and 'centerFreq1HPPM'  in magritek_params and magritek_params["nucleus1"]!=magritek_params["nucleus2"]:
        modified_acqu_content.append(f'##$BF1= {magritek_params["b1Freq"]*(1.0-magritek_params["centerFreq1HPPM"]*1e-6)}\n')
    elif line.startswith('##$BF2=') and 'b1Freq2'  and 'centerFreq13CPPM'  in magritek_params and magritek_params["nucleus2"]=='13C':
        modified_acqu_content.append(f'##$BF2= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreq13CPPM"]*1e-6)}\n')
    elif line.startswith('##$BF2=') and 'b1Freq2'  and 'centerFreq15NPPM'  in magritek_params and magritek_params["nucleus2"]=='15N':
        modified_acqu_content.append(f'##$BF2= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreq15NPPM"]*1e-6)}\n')
    elif line.startswith('##$SW_h=') and 'bandwidth' in magritek_params:
        modified_acqu_content.append(f'##$SW_h= {magritek_params["bandwidth"]}\n')
    elif line.startswith('##$SW=') and 'bandwidth' and 'b1Freq' in magritek_params:
        modified_acqu_content.append(f'##$SW= {magritek_params["bandwidth"]/magritek_params["b1Freq"]}\n')
    else:
        modified_acqu_content.append(line)

# Write the modified content back to the 'acqu' file.
with open(acqu_file_path, 'w') as f:
    f.writelines(modified_acqu_content)

# Read the content of the 'acqus' file.
with open(acqus_file_path, 'r') as f:
    acqus_content = f.readlines()

# Modify the content of the 'acqus' file
modified_acqus_content = []
for line in acqus_content:
    if line.startswith('##$TD='):
        modified_acqus_content.append(f'##$TD= {data_to_save_complex.shape[1]*2}\n')
    elif line.startswith('##$DIGTYP='):
        modified_acqus_content.append('##$DIGTYP= 17\n')
    elif line.startswith('##$DTYPA='):
        modified_acqus_content.append('##$DTYPA= 2\n')
    elif line.startswith('##$NUC1=') and 'nucleus1' in magritek_params:
        modified_acqus_content.append(f'##$NUC1= <{magritek_params["nucleus1"]}>\n')
    elif line.startswith('##$NUC2=') and 'nucleus2' in magritek_params:
        modified_acqus_content.append(f'##$NUC2= <{magritek_params["nucleus2"]}>\n')
    elif line.startswith('##$SFO1=') and 'b1Freq' in magritek_params:
        modified_acqus_content.append(f'##$SFO1= {magritek_params["b1Freq"]}\n')
    elif line.startswith('##$SFO2=') and 'b1Freq2' in magritek_params:
        modified_acqus_content.append(f'##$SFO2= {magritek_params["b1Freq2"]}\n')
    elif line.startswith('##$BF1=') and 'b1Freq' and 'centerFreqPPM'  in  magritek_params and magritek_params["nucleus1"]==magritek_params["nucleus2"]:
        modified_acqus_content.append(f'##$BF1= {magritek_params["b1Freq"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$BF2=') and 'b1Freq2' and 'centerFreqPPM'  in  magritek_params and magritek_params["nucleus1"]==magritek_params["nucleus2"]:
        modified_acqus_content.append(f'##$BF2= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$BF1=') and 'b1Freq' and 'centerFreq1HPPM'  in  magritek_params and magritek_params["nucleus1"]!=magritek_params["nucleus2"]:
        modified_acqus_content.append(f'##$BF1= {magritek_params["b1Freq"]*(1.0-magritek_params["centerFreq1HPPM"]*1e-6)}\n')
    elif line.startswith('##$BF2=') and 'b1Freq2' and 'centerFreq13CPPM'  in  magritek_params and magritek_params["nucleus2"]=='13C':
        modified_acqus_content.append(f'##$BF2= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreq13CPPM"]*1e-6)}\n')
    elif line.startswith('##$BF2=') and 'b1Freq2' and 'centerFreq15NPPM'  in  magritek_params and magritek_params["nucleus2"]=='15N':
        modified_acqus_content.append(f'##$BF2= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreq15NPPM"]*1e-6)}\n')
    elif line.startswith('##$SW_h=') and 'bandwidth' in magritek_params:
        modified_acqus_content.append(f'##$SW_h= {magritek_params["bandwidth"]}\n')
    elif line.startswith('##$SW=') and 'bandwidth' and 'b1Freq' in magritek_params:
        modified_acqus_content.append(f'##$SW= {magritek_params["bandwidth"]/magritek_params["b1Freq"]}\n')
    else:
        modified_acqus_content.append(line)

# Write the modified content back to the 'acqus' file.
with open(acqus_file_path, 'w') as f:
    f.writelines(modified_acqus_content)

print("Acqu and acqus files modified successfully.")

# Define the paths to the 'acqu2' and 'acqu2s' files within the '1001' folder.
acqu2_file_path =  destination_folder+'/acqu2'
acqu2s_file_path = destination_folder+'/acqu2s'

# Read the content of the 'acqu2' file.
with open(acqu2_file_path, 'r') as f:
    acqu2_content = f.readlines()

# Modify the content of the 'acqu2' file
modified_acqu2_content = []
for line in acqu2_content:
    if line.startswith('##$TD='):
        modified_acqu2_content.append(f'##$TD= {data_to_save_complex.shape[0]}\n')
    elif line.startswith('##$DIGTYP='):
        modified_acqu2_content.append('##$DIGTYP= 17\n')
    elif line.startswith('##$DTYPA='):
        modified_acqu2_content.append('##$DTYPA= 2\n')
    elif line.startswith('##$FnMODE=') and quad==1:
        modified_acqu2_content.append('##$FnMODE= 0\n')
    elif line.startswith('##$FnMODE=') and quad==2:
        modified_acqu2_content.append('##$FnMODE= 4\n')
    elif line.startswith('##$FnMODE=') and quad==3:
        modified_acqu2_content.append('##$FnMODE= 6\n')
    elif line.startswith('##$NUC1=') and 'nucleus2' in magritek_params:
        modified_acqu2_content.append(f'##$NUC1= <{magritek_params["nucleus2"]}>\n')
    elif line.startswith('##$SFO1=') and 'b1Freq2' in magritek_params:
        modified_acqu2_content.append(f'##$SFO1= {magritek_params["b1Freq2"]}\n')
    elif line.startswith('##$BF1=') and 'b1Freq2' and 'centerFreqPPM'   in  magritek_params and magritek_params["nucleus1"]==magritek_params["nucleus2"]:
        modified_acqu2_content.append(f'##$BF1= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$BF1=') and 'b1Freq2' and 'centerFreq13CPPM'   in  magritek_params and magritek_params["nucleus2"]=='13C':
        modified_acqu2_content.append(f'##$BF1= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreq13CPPM"]*1e-6)}\n')
    elif line.startswith('##$BF1=') and 'b1Freq2' and 'centerFreq15NPPM'   in  magritek_params and magritek_params["nucleus2"]=='15N':
        modified_acqu2_content.append(f'##$BF1= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreq15NPPM"]*1e-6)}\n')
    elif line.startswith('##$SW_h=') and 'bandwidth2' in magritek_params:
        modified_acqu2_content.append(f'##$SW_h= {magritek_params["bandwidth2"]}\n')
    elif line.startswith('##$SW=') and 'bandwidthf1PPM' in magritek_params:
        modified_acqu2_content.append(f'##$SW= {magritek_params["bandwidthf1PPM"]}\n')
    else:
        modified_acqu2_content.append(line)

# Write the modified content back to the 'acqu2' file.
with open(acqu2_file_path, 'w') as f:
    f.writelines(modified_acqu2_content)

# Read the content of the 'acqu2s' file.
with open(acqu2s_file_path, 'r') as f:
    acqu2s_content = f.readlines()

# Modify the content of the 'acqu2s' file
modified_acqu2s_content = []
for line in acqu2s_content:
    if line.startswith('##$TD='):
        modified_acqu2s_content.append(f'##$TD= {data_to_save_complex.shape[0]}\n')
    elif line.startswith('##$DIGTYP='):
        modified_acqu2s_content.append('##$DIGTYPA= 0\n')
    elif line.startswith('##$DTYPA='):
        modified_acqu2s_content.append('##$DTYPA= 2\n')
    elif line.startswith('##$FnMODE=') and quad==1:
        modified_acqu2s_content.append('##$FnMODE= 0\n')
    elif line.startswith('##$FnMODE=') and quad==2:
        modified_acqu2s_content.append('##$FnMODE= 4\n')
    elif line.startswith('##$FnMODE=') and quad==3:
        modified_acqu2s_content.append('##$FnMODE= 6\n')
    elif line.startswith('##$NUC1=') and 'nucleus2' in magritek_params:
        modified_acqu2s_content.append(f'##$NUC1= <{magritek_params["nucleus2"]}>\n')
    elif line.startswith('##$SFO1=') and 'b1Freq2' in magritek_params:
        modified_acqu2s_content.append(f'##$SFO1= {magritek_params["b1Freq2"]}\n')
    elif line.startswith('##$BF1=') and 'b1Freq2'  and 'centerFreqPPM'  in  magritek_params and magritek_params["nucleus1"]==magritek_params["nucleus2"]:
        modified_acqu2s_content.append(f'##$BF1= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$BF1=') and 'b1Freq2'  and 'centerFreq13CPPM'  in  magritek_params and magritek_params["nucleus2"]=='13C':
        modified_acqu2s_content.append(f'##$BF1= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreq13CPPM"]*1e-6)}\n')
    elif line.startswith('##$BF1=') and 'b1Freq2'  and 'centerFreq15NPPM'  in  magritek_params and magritek_params["nucleus2"]=='15N':
        modified_acqu2s_content.append(f'##$BF1= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreq15NPPM"]*1e-6)}\n')
    elif line.startswith('##$SW_h=') and 'bandwidth2' in magritek_params:
        modified_acqu2s_content.append(f'##$SW_h= {magritek_params["bandwidth2"]}\n')
    elif line.startswith('##$SW=') and 'bandwidthf1PPM' in magritek_params:
        modified_acqu2s_content.append(f'##$SW= {magritek_params["bandwidthf1PPM"]}\n')
    else:
        modified_acqu2s_content.append(line)

# Write the modified content back to the 'acqu2s' file.
with open(acqu2s_file_path, 'w') as f:
    f.writelines(modified_acqu2s_content)

print("Acqu2 and acqu2s files modified successfully.")

import os

# Define the paths to the 'proc', 'proc2' and 'proc2s' files within the '1001/pdata/1' folder.
proc_file_path =  destination_folder+'/pdata/1/proc'
procs_file_path =  destination_folder+'/pdata/1/procs'
proc2_file_path =  destination_folder+'/pdata/1/proc2'
proc2s_file_path = destination_folder+'/pdata/1/proc2s'


# Read the content of the 'proc' file.
with open(proc_file_path, 'r') as f:
    proc_content = f.readlines()

# Modify the content of the 'proc' file
modified_proc_content = []
for line in proc_content:
    if line.startswith('##$REVERSE='):
        modified_proc_content.append('##$REVERSE= yes\n')
    elif line.startswith('##$SI='):
        modified_proc_content.append(f'##$SI= {data_to_save_complex.shape[1]*2}\n')
    elif line.startswith('##$DTYPA='):
        modified_proc_content.append('##$DTYPA= 2\n')
    elif line.startswith('##$OFFSET=') and 'centerFreqPPM'  in  magritek_params and magritek_params["nucleus1"]==magritek_params["nucleus2"]:
        modified_proc_content.append(f'##$OFFSET= {magritek_params["centerFreqPPM"]}\n')
    elif line.startswith('##$OFFSET=') and 'centerFreq1HPPM'  in  magritek_params and magritek_params["nucleus1"]!=magritek_params["nucleus2"]:
        modified_proc_content.append(f'##$OFFSET= {magritek_params["centerFreq1HPPM"]}\n')
    elif line.startswith('##$SF=') and 'b1Freq' in magritek_params:
        modified_proc_content.append(f'##$SF= {magritek_params["b1Freq"]}\n')
    elif line.startswith('##$SW_p=') and 'bandwidth' in magritek_params:
        modified_proc_content.append(f'##$SW_p= {magritek_params["bandwidth"]}\n')
    else:
        modified_proc_content.append(line)

# Write the modified content back to the 'proc' file.
with open(proc_file_path, 'w') as f:
    f.writelines(modified_proc_content)

# Read the content of the 'procs' file.
with open(procs_file_path, 'r') as f:
    procs_content = f.readlines()

# Modify the content of the 'proc' file
modified_procs_content = []
for line in procs_content:
    if line.startswith('##$REVERSE='):
        modified_procs_content.append('##$REVERSE= yes\n')
    elif line.startswith('##$SI='):
        modified_procs_content.append(f'##$SI= {data_to_save_complex.shape[1]*2}\n')
    elif line.startswith('##$DTYPA='):
        modified_procs_content.append('##$DTYPA= 2\n')
    elif line.startswith('##$OFFSET=') and 'centerFreqPPM'  in  magritek_params and magritek_params["nucleus1"]==magritek_params["nucleus2"]:
        modified_procs_content.append(f'##$OFFSET= {magritek_params["centerFreqPPM"]}\n')
    elif line.startswith('##$OFFSET=') and 'centerFreq1HPPM'  in  magritek_params and magritek_params["nucleus1"]!=magritek_params["nucleus2"]:
        modified_procs_content.append(f'##$OFFSET= {magritek_params["centerFreq1HPPM"]}\n')
    elif line.startswith('##$SF=') and 'b1Freq' in magritek_params:
        modified_procs_content.append(f'##$SF= {magritek_params["b1Freq"]}\n')
    elif line.startswith('##$SW_p=') and 'bandwidth' in magritek_params:
        modified_procs_content.append(f'##$SW_p= {magritek_params["bandwidth"]}\n')
    else:
        modified_procs_content.append(line)

# Write the modified content back to the 'proc' file.
with open(procs_file_path, 'w') as f:
    f.writelines(modified_procs_content)

# Read the content of the 'proc2' file.
with open(proc2_file_path, 'r') as f:
    proc2_content = f.readlines()

# Modify the content of the 'proc2' file
modified_proc2_content = []
for line in proc2_content:
    if line.startswith('##$REVERSE='):
        modified_proc2_content.append('##$REVERSE= yes\n')
    elif line.startswith('##$SI='):
        modified_proc2_content.append(f'##$SI= {data_to_save_complex.shape[0]*quad}\n')
    elif line.startswith('##$DTYPA='):
        modified_proc2_content.append('##$DTYPA= 2\n')
    elif line.startswith('##$OFFSET=') and 'centerFreqPPM'  in  magritek_params and magritek_params["nucleus1"]==magritek_params["nucleus2"]:
        modified_proc2_content.append(f'##$OFFSET= {magritek_params["centerFreqPPM"]}\n')
    elif line.startswith('##$OFFSET=') and 'centerFreq13CPPM'  in  magritek_params and magritek_params["nucleus2"]=='13C':
        modified_proc2_content.append(f'##$OFFSET= {magritek_params["centerFreq13CPPM"]}\n')
    elif line.startswith('##$OFFSET=') and 'centerFreq15NPPM'  in  magritek_params and magritek_params["nucleus2"]=='15N':
        modified_proc2_content.append(f'##$OFFSET= {magritek_params["centerFreq15NPPM"]}\n')
    elif line.startswith('##$SW_p=') and 'bandwidth2' in magritek_params:
        modified_proc2_content.append(f'##$SW_p= {magritek_params["bandwidth2"]}\n')
    elif line.startswith('##$SF=') and 'b1Freq2' in magritek_params:
        modified_proc2_content.append(f'##$SF= {magritek_params["b1Freq2"]}\n')
    elif line.startswith('##$PH_mod=') and quad==1:
        modified_proc2_content.append(f'##$PH_mod=2\n')
    elif line.startswith('##$MC2=') and quad==1:
        modified_proc2_content.append(f'##$MC2=0\n')
    else:
        modified_proc2_content.append(line)

# Write the modified content back to the 'proc2' file.
with open(proc2_file_path, 'w') as f:
    f.writelines(modified_proc2_content)

# Read the content of the 'proc2s' file.
with open(proc2s_file_path, 'r') as f:
    proc2s_content = f.readlines()

# Modify the content of the 'proc2s' file
modified_proc2s_content = []
for line in proc2s_content:
    if line.startswith('##$REVERSE='):
        modified_proc2s_content.append('##$REVERSE= yes\n')
    elif line.startswith('##$SI='):
        modified_proc2s_content.append(f'##$SI= {data_to_save_complex.shape[0]*quad}\n')
    elif line.startswith('##$DTYPA='):
        modified_proc2s_content.append('##$DTYPA= 2\n')
    elif line.startswith('##$OFFSET=') and 'centerFreqPPM' in  magritek_params and magritek_params["nucleus1"]==magritek_params["nucleus2"]:
        modified_proc2s_content.append(f'##$OFFSET= {magritek_params["centerFreqPPM"]}\n')
    elif line.startswith('##$OFFSET=') and 'centerFreq13CPPM' in  magritek_params and magritek_params["nucleus2"]=='13C':
        modified_proc2s_content.append(f'##$OFFSET= {magritek_params["centerFreq13CPPM"]}\n')
    elif line.startswith('##$OFFSET=') and 'centerFreq15NPPM' in  magritek_params and magritek_params["nucleus2"]=='15N':
        modified_proc2s_content.append(f'##$OFFSET= {magritek_params["centerFreq15NPPM"]}\n')
    elif line.startswith('##$SW_p=') and 'bandwidth2' in magritek_params:
        modified_proc2s_content.append(f'##$SW_p= {magritek_params["bandwidth2"]}\n')
    elif line.startswith('##$SF=') and 'b1Freq2' in magritek_params:
        modified_proc2s_content.append(f'##$SF= {magritek_params["b1Freq2"]}\n')
    elif line.startswith('##$MC2=') and quad==1:
        modified_proc2s_content.append(f'##$MC2=0\n')
    else:
        modified_proc2s_content.append(line)

# Write the modified content back to the 'proc2s' file.
with open(proc2s_file_path, 'w') as f:
    f.writelines(modified_proc2s_content)

print("Proc, Proc2 and proc2s files modified successfully.")

Folder 'template' copied to 'example/1003'
Time-domain data (FID) saved to example/1003/ser.
Acqu and acqus files modified successfully.
Acqu2 and acqu2s files modified successfully.
Proc, Proc2 and proc2s files modified successfully.
